In [1]:
import os, openai
from config import apikey, pinecone_apikey

In [2]:
from langchain.document_loaders import DirectoryLoader

directory = "/Users/dipit/ChatGPT/LangChain_Streamlit/Files"

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents
documents = load_docs(directory)
len(documents)

2

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, chunk_size = 1000, chunk_overlap = 20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, 
                                                   chunk_overlap = chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs
docs = split_docs(documents)
len(docs)

13

In [4]:
docs[0].page_content

'What are reasons behind gastric trouble or formation of gases in stomach?\n\nThere can be several reasons behind gastric trouble or the formation of gases in the stomach. Some of the common causes include:\n\n1. Poor diet: Consuming foods that are difficult to digest, such as fatty or processed foods, can lead to gastric trouble.\n\n2. Overeating: Eating large meals or eating too quickly can cause the stomach to produce excess gas.\n\n3. Food intolerances: Some individuals may have difficulty digesting certain types of foods, such as lactose or gluten, leading to the formation of gas.\n\n4. Swallowed air: Eating or drinking too quickly, chewing gum, smoking, or using straws can cause excess air to be swallowed, leading to gas in the stomach.\n\n5. Gastrointestinal disorders: Conditions like irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), or gastroesophageal reflux disease (GERD) can cause gastric trouble and gas formation.'

In [5]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model_name = "ada", openai_api_key=apikey
)
query_result = embeddings.embed_query("Hello World")
len(query_result)

/Users/dipit/anaconda3/envs/gpt/lib/python3.10/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! model_name is not default parameter.
                    model_name was transferred to model_kwargs.
                    Please confirm that model_name is what you intended.
  warnings.warn(


1536

In [6]:
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=pinecone_apikey,
    environment='us-west1-gcp-free'
)
index = 'chatbot'
index = Pinecone.from_documents(docs, embeddings, index_name = index)

/Users/dipit/anaconda3/envs/gpt/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
def get_similar_docs(query, k=1, score=False):
    if score:
        similar_docs = index.similarity_search_with_score(query, k=k)
    else:
        similar_docs = index.similarity_search(query, k=k)
    return similar_docs

query = "overeating cause gastric trouble"
print(get_similar_docs(query, score=True))

[(Document(page_content='What are reasons behind gastric trouble or formation of gases in stomach?\n\nThere can be several reasons behind gastric trouble or the formation of gases in the stomach. Some of the common causes include:\n\n1. Poor diet: Consuming foods that are difficult to digest, such as fatty or processed foods, can lead to gastric trouble.\n\n2. Overeating: Eating large meals or eating too quickly can cause the stomach to produce excess gas.\n\n3. Food intolerances: Some individuals may have difficulty digesting certain types of foods, such as lactose or gluten, leading to the formation of gas.\n\n4. Swallowed air: Eating or drinking too quickly, chewing gum, smoking, or using straws can cause excess air to be swallowed, leading to gas in the stomach.\n\n5. Gastrointestinal disorders: Conditions like irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), or gastroesophageal reflux disease (GERD) can cause gastric trouble and gas formation.', metadata={'source'

In [8]:
from langchain.llms import OpenAI
model_name = 'gpt-3.5-turbo'
llm = OpenAI(model_name = model_name, openai_api_key=apikey)

/Users/dipit/anaconda3/envs/gpt/lib/python3.10/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/dipit/anaconda3/envs/gpt/lib/python3.10/site-packages/langchain/llms/openai.py:801: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    similar_docs = get_similar_docs(query)
    answer = chain.run(input_documents = similar_docs, question=query)
    return answer

query = "One of the reasons for gastric troubles"
get_answer(query)

'Poor diet is one of the reasons for gastric troubles.'

In [10]:
query = "Homeopathic medicines for curing gastric troubles"
print(get_answer(query))

Carbo vegetabilis, Lycopodium, Nux vomica, and China officinalis are commonly used homeopathic remedies for relieving various gastric troubles such as bloating, belching, flatulence, and abdominal discomfort.
